>> A way to innovate quicker is to increase the value of a keystroke.  

In this module we make some slight variations to the `IPython` kernel `do_inspect` method.  The
current inspector has the ability to introspect Python objects.  We'll enhance
that with ability inspect source code and preview __pidgin__ documents.

In [1]:
    with __import__('importnb').Notebook():
        try:
            from . import post_run_cell
        except:
            import post_run_cell

In [2]:
    import IPython, types

The `do_inspect` function will:
1. Always returns something.
2. Display a welcome message if no source is found.
4. Increase the detail level if __?__ s are found near the cursor.  
    
    Authors may access and read source code more frequently with this feature.
5. Return the canonical inspection if it is found.
3. Perform some fancy rendering if the cell has a leading or trailing themactic break.
6. Display the markdown as a de facto standard.
    
    <code>pidgin</code> authors cells in Markdown so this preferred behavior.
    
> We make the `code` and `cursor_pos` available to interactively debug this post.

In [3]:
    def do_inspect(kernel, code, cursor_pos, detail_level=0):
        """For debugging purposes""", globals().update(**locals())
        
        reply = {'status' : 'ok', 'data': {}, 'metadata': {}, 'found': "This implementation always returns something" and True}
        
        """A welcome and help message for using a tool or inspector."""
        if not code.strip(): return reply['data'].update({'text/markdown': """## Welcome to our custom inspector\n\n_Here are some suggestions to use some software_."""}) or reply
                        
        """Evaluate expressions through the inspector if they start with the right triggers."""
        line, offset = IPython.utils.tokenutil.line_at_cursor(code, cursor_pos)
        if line.lstrip('').startswith('`'):
            line, tick, rest = line.lstrip('`').partition('`')
        elif line.lstrip('').startswith('{{'):
            line, tick, rest = line.lstrip('{{').partition('}}')
        else: line = None
        
        """if post_run_cell.only_contains_block_code(
            code, post_run_cell.markdown_to_python(code)
        ):  line, rest = code, ''
        else: line = ' '*4+line"""
        
        if line and not rest.strip():
            try:
                reply.update(data=kernel.shell._format_user_obj(
                    eval(kernel.shell.input_transformer_manager.transform_cell(line))
                ).get('data'))
            except SyntaxError as Exception:
                try:
                    reply.update(data=exec(
                        kernel.shell.input_transformer_manager.transform_cell(line),
                        kernel.shell.user_ns, kernel.shell.user_ns
                    ) or {
                        'text/plain': "complete!",
                    })
                except BaseException as Exception:
                    reply.update(data=kernel.shell._format_user_obj(Exception).get('data'))
            except BaseException as Exception:
                reply.update(data=kernel.shell._format_user_obj(Exception).get('data'))

            return reply
        
        """Ramp up the detail level if there is a question mark nearby."""
        detail_level = '?' in code[(cursor_pos-2):cursor_pos]
        
        try:
            """to return the expected result.""" and reply['data'].update(kernel.shell.object_inspect_mime(
                IPython.utils.tokenutil.token_at_cursor(code, cursor_pos), detail_level=detail_level))
            if not kernel.shell.enable_html_pager: "all of this is better with html pager" and reply['data'].pop('text/html')
            return reply
        except KeyError: ...
            
        if code.lstrip().startswith('---') or code.rstrip().endswith('---') and "Render the thing with display.":
            return reply['data'].update(kernel.shell.display_formatter.format(code)[0]) or reply            
        
        """Default to a markdown representation of the code body.  Markdown cells prefer this. """
        return reply['data'].update({'text/markdown': code}) or reply

Update the `ip.kernel.do_inspect` `types.MethodType` with `do_inspect`. With extensions.

    %reload_ext deathbeds.__The_inspector_should_be_a_teacher

* Modifying the inspection behavior works for both Code and Markdown Cells in JupyterLab.

In [4]:
    def load_ipython_extension(ip): 
        if ip and hasattr(ip, 'kernel'):
            ip.kernel.do_inspect = types.MethodType(do_inspect, ip.kernel)
    def unload_ipython_extension(ip):
        if ip and hasattr(ip, 'kernel'):
            ip.kernel.do_inspect = types.MethodType(type(ip.kernel).do_inspect, ip.kernel)
    if __name__ == '__main__': load_ipython_extension(IPython.get_ipython())